In [8]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from sklearn.model_selection import train_test_split
import random

import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="PIL")

In [9]:
base_dir = '../data/Category_and_Attribute_Prediction_Benchmark'

# Load category annotations
category_img = pd.read_csv('../data/Category_and_Attribute_Prediction_Benchmark/Anno_coarse/list_category_img.txt', sep='\s+', skiprows=2, header=None)
category_img.columns = ['image_name', 'category_label']

# Load category names
category_names = pd.read_csv('../data/Category_and_Attribute_Prediction_Benchmark/Anno_coarse/list_category_cloth.txt', sep='\s+', skiprows=1, header=None)
category_names.columns = ['category_name', 'category_type']

# Load attribute names 
attribute_names = pd.read_csv('../data/Category_and_Attribute_Prediction_Benchmark/Anno_fine/list_attr_cloth.txt', sep='\s+', skiprows=1, header=None)
attribute_names.columns = ['attribute_name', 'attribute_type']

# Merge category names with image annotations
category_img = category_img.merge(category_names, left_on='category_label', right_index=True)

# Load attribute annotations
attr_img = pd.read_csv('../data/Category_and_Attribute_Prediction_Benchmark/Anno_fine/list_attr_img_update.txt', sep='\s+', skiprows=2, header=None)
attr_img.columns = ['image_name'] + [f'attr_{i+1}' for i in range(attr_img.shape[1] - 1)]

category_img['image_name'] = category_img['image_name'].str.replace('img/', 'img_highres/')
attr_img['image_name'] = attr_img['image_name'].str.replace('img_highres_subset/', 'img_highres/')

print(category_img['image_name'].head())
print(attr_img['image_name'].head())

# Merge with category data
dataset = pd.merge(category_img, attr_img, on='image_name')
print(len(dataset))

0    img_highres/Sheer_Pleated-Front_Blouse/img_000...
1    img_highres/Sheer_Pleated-Front_Blouse/img_000...
2    img_highres/Sheer_Pleated-Front_Blouse/img_000...
3    img_highres/Sheer_Pleated-Front_Blouse/img_000...
4    img_highres/Sheer_Pleated-Front_Blouse/img_000...
Name: image_name, dtype: object
0    img_highres/Striped_Denim_Shorts/img_00000037.jpg
1       img_highres/Rose_Print_Shorts/img_00000058.jpg
2    img_highres/Embroidered_High-Neck_Blouse/img_0...
3     img_highres/Tie-Dye_Cami_Romper/img_00000001.jpg
4    img_highres/Favorite_Scoop_Neck_Tank/img_00000...
Name: image_name, dtype: object
20000


# Subset the Dataset from list_eval_partition.txt
Split them into train, val and test

In [10]:
# Sample a subset of the dataset
eval_partition = pd.read_csv(f'{base_dir}/Eval/list_eval_partition.txt', sep='\s+', skiprows=2, header=None)
eval_partition.columns = ['image_name', 'evaluation_status']

eval_partition['image_name'] = eval_partition['image_name'].str.replace('img/', 'img_highres/')

# Merge evaluation partition with dataset
dataset = pd.merge(dataset, eval_partition, on='image_name')

# Split into train, val, and test datasets
train_data = dataset[dataset['evaluation_status'] == 'train']
val_data = dataset[dataset['evaluation_status'] == 'val']
test_data = dataset[dataset['evaluation_status'] == 'test']

# Print the lengths of each dataset for verification
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"Testing samples: {len(test_data)}")

Training samples: 12002
Validation samples: 2001
Testing samples: 5997


## Split training data into 8 chunks

In [14]:
# Split into 8 chunks
chunks = np.array_split(train_data, 8)

# ImageDataGenerator setup
datagen = ImageDataGenerator(rescale=1.0 / 255)

# Define the model
def create_xception_model(num_attributes):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Category output
    category_output = Dense(len(category_names), activation='softmax', name='category_output')(x)

    # Attribute output
    attribute_output = Dense(num_attributes, activation='sigmoid', name='attribute_output')(x)

    # Build the model
    model = Model(inputs=base_model.input, outputs=[category_output, attribute_output])

    # Compile with SGD optimizer
    optimizer = SGD(learning_rate=1e-4, momentum=0.9)
    model.compile(
        optimizer=optimizer,
        loss={
            'category_output': 'categorical_crossentropy',
            'attribute_output': 'binary_crossentropy',
        },
        metrics={
            'category_output': 'accuracy',
            'attribute_output': 'accuracy',
        },
    )
    return model


/Users/dalenlim/cs310/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


# Pre-train the model

In [15]:
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array

class MultiOutputDataGenerator(Sequence):
    def __init__(self, dataframe, directory, x_col, y_cols, batch_size, target_size):
        self.dataframe = dataframe
        self.directory = directory
        self.x_col = x_col
        self.y_cols = y_cols
        self.batch_size = batch_size
        self.target_size = target_size
        self.indexes = np.arange(len(dataframe))

    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        # Get the batch indices
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        
        # Get the batch dataframe
        batch_data = self.dataframe.iloc[batch_indexes]

        # Process the inputs and outputs
        X, y_category, y_attributes = [], [], []
        for _, row in batch_data.iterrows():
            # Load and preprocess the image
            img_path = f"{self.directory}/{row[self.x_col]}"
            img = load_img(img_path, target_size=self.target_size)
            img = img_to_array(img) / 255.0  # Normalize to [0, 1]
            X.append(img)
            
            # Extract labels
            y_category.append(row['category_label'])
            y_attributes.append(row[self.y_cols].values.astype(float))
        
        # Convert to NumPy arrays
        X = np.array(X)
        y_category = tf.keras.utils.to_categorical(y_category, num_classes=len(category_names))
        y_attributes = np.array(y_attributes)

        return X, {'category_output': y_category, 'attribute_output': y_attributes}


In [17]:
# Define columns for labels
attribute_cols = [f'attr_{i+1}' for i in range(attr_img.shape[1] - 1)]

# Number of attributes
num_attributes = len(attribute_cols)

print(num_attributes)

# Pre-train on each chunk for 15 epochs
for i, chunk in enumerate(chunks):
    print(f"Pre-training on chunk {i+1}/8")

    train_classes = set(chunk['category_name'])
    val_classes = set(val_data['category_name'])

    common_classes = train_classes.intersection(val_classes)
    print(f"Common classes: {len(common_classes)}")

    # Filter datasets to include only common classes
    chunk = chunk[chunk['category_name'].isin(common_classes)]
    val_data = val_data[val_data['category_name'].isin(common_classes)]
    
    # Create custom training generator
    train_gen = MultiOutputDataGenerator(
        dataframe=chunk,
        directory='../data/Category_and_Attribute_Prediction_Benchmark/Img/',
        x_col='image_name',
        y_cols=attribute_cols,
        batch_size=32,
        target_size=(100, 100)
    )
    
    # Create custom validation generator
    val_gen = MultiOutputDataGenerator(
        dataframe=val_data,
        directory='../data/Category_and_Attribute_Prediction_Benchmark/Img/',
        x_col='image_name',
        y_cols=attribute_cols,
        batch_size=64,
        target_size=(100, 100)
    )
    
    # Create and train the model
    model = create_xception_model(num_attributes)
    model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=15,
        steps_per_epoch=len(train_gen),
        validation_steps=len(val_gen)
    )
    
    # Save weights after training each chunk
    model.save_weights(f"xception_weights_chunk_{i+1}.weights.h5")

26
Pre-training on chunk 1/8
Common classes: 13


/Users/dalenlim/cs310/venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 26), output.shape=(None, 51)

In [18]:
# Pre-train on each chunk for 15 epochs
for i, chunk in enumerate(chunks):
    print(f"Pre-training on chunk {i+1}/8")
    
    # Prepare the target columns for multi-output
    y_columns = ['category_label'] + [f'attr_{i+1}' for i in range(attr_img.shape[1] - 1)]

    # Ensure targets are included as separate columns in the generator
    chunk_targets = chunk[y_columns].copy()
    
    # Data generator for the current chunk
    train_gen = datagen.flow_from_dataframe(
        dataframe=chunk,
        directory='../data/Category_and_Attribute_Prediction_Benchmark/Img/',
        x_col='image_name',
        y_col=y_columns,
        target_size=(100, 100),
        batch_size=32,
        class_mode='multi_output'
    )

    val_gen = datagen.flow_from_dataframe(
        dataframe=val_data,
        directory='../data/Category_and_Attribute_Prediction_Benchmark/Img/',
        x_col='image_name',
        y_col=y_columns,
        target_size=(100, 100),
        batch_size=64,
        class_mode='multi_output'
    )
    
    # Create and train the model
    model = create_xception_model()
    model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=15,
        steps_per_epoch=len(train_gen),
        validation_steps=len(val_gen)
    )
    
    # Save weights after training each chunk
    model.save_weights(f"xception_weights_chunk_{i+1}.weights.h5")


Pre-training on chunk 1/8
Found 1501 validated image filenames.
Found 2001 validated image filenames.


/Users/dalenlim/cs310/venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


TypeError: `output_signature` must contain objects that are subclass of `tf.TypeSpec` but found <class 'list'> which is not.

In [ ]:
# Randomly select one chunk
selected_chunk_idx = random.randint(0, 7)
selected_chunk = chunks[selected_chunk_idx]
print(f"Selected chunk: {selected_chunk_idx + 1}")

# Train the model on the selected chunk for 100 epochs
train_gen = datagen.flow_from_dataframe(
    dataframe=selected_chunk,
    directory='../data/Category_and_Attribute_Prediction_Benchmark/Img/',
    x_col='image_name',
    y_col=['category_label'] + [f'attr_{i+1}' for i in range(attr_img.shape[1] - 1)],
    target_size=(100, 100),
    batch_size=64,
    class_mode='multi_output'
)

val_gen = datagen.flow_from_dataframe(
    dataframe=val_data,
    directory='../data/Category_and_Attribute_Prediction_Benchmark/Img/',
    x_col='image_name',
    y_col=['category_label'] + [f'attr_{i+1}' for i in range(attr_img.shape[1] - 1)],
    target_size=(100, 100),
    batch_size=64,
    class_mode='multi_output'
)
    
model = create_xception_model()
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen)
)

# Save the final model
model.save('xception_final_model.weights.h5')

# Evaluate on test data
test_gen = datagen.flow_from_dataframe(
    dataframe=test_data,
    directory='../data/Category_and_Attribute_Prediction_Benchmark/Img/',
    x_col='image_name',
    y_col=['category_label'] + [f'attr_{i+1}' for i in range(attr_img.shape[1] - 1)],
    target_size=(100, 100),
    batch_size=64,
    class_mode='multi_output'
)

test_results = model.evaluate(test_gen, steps=len(test_gen))
print(f"Test Results: {test_results}")